In [125]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [126]:
import pandas as pd

# Step 1: Load both the standings data and the driver data
driver_standings = pd.read_csv('/content/driver_standings.csv')
drivers = pd.read_csv('/content/drivers.csv')
circuits = pd.read_csv('/content/circuits.csv')
constructors = pd.read_csv('/content/constructors.csv')
constructor_results = pd.read_csv('/content/constructor_results.csv')
constructor_standings = pd.read_csv('/content/constructor_standings.csv')
lap_times = pd.read_csv('/content/lap_times.csv')
qualifying = pd.read_csv('/content/qualifying.csv')
races = pd.read_csv('/content/races.csv')
results = pd.read_csv('/content/results.csv')
seasons = pd.read_csv('/content/seasons.csv')
status = pd.read_csv('/content/status.csv')


df = pd.merge(results, races, on='raceId')

# Step 3: View the result
# You will now see columns for driver names, nationality, etc
df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,18,1,1,22,1,1,1,1,10.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,18,2,2,3,5,2,2,2,8.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,18,3,3,7,7,3,3,3,6.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,18,4,4,5,11,4,4,4,5.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,18,5,1,23,3,5,5,5,4.0,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [127]:
# merging more
df = pd.merge(df, drivers, on='driverId', how='left')
df = pd.merge(df, constructors, on='constructorId', how='left')
df = pd.merge(df, status, on='statusId', how='left')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27058 entries, 0 to 27057
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         27058 non-null  int64  
 1   raceId           27058 non-null  int64  
 2   driverId         27058 non-null  int64  
 3   constructorId    27058 non-null  int64  
 4   number_x         27058 non-null  object 
 5   grid             27058 non-null  int64  
 6   position         27058 non-null  object 
 7   positionText     27058 non-null  object 
 8   positionOrder    27058 non-null  int64  
 9   points           27058 non-null  float64
 10  laps             27058 non-null  int64  
 11  time_x           27058 non-null  object 
 12  milliseconds     27058 non-null  object 
 13  fastestLap       27058 non-null  object 
 14  rank             27058 non-null  object 
 15  fastestLapTime   27058 non-null  object 
 16  fastestLapSpeed  27058 non-null  object 
 17  statusId    

In [128]:
# --- Step 2: Perform the core merges ---

# Start with results and merge with races to get race details (year, circuitId)
df = pd.merge(results, races, on='raceId', how='left')

# Merge with drivers to get driver details
df = pd.merge(df, drivers, on='driverId', how='left')

# Merge with constructors to get constructor details
df = pd.merge(df, constructors, on='constructorId', how='left')

# Merge with status to understand the finishing status
df = pd.merge(df, status, on='statusId', how='left')

# Merge with qualifying to get qualifying position
df = pd.merge(df, qualifying, on=['raceId', 'driverId'], how='left')

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27058 entries, 0 to 27057
Data columns (total 55 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         27058 non-null  int64  
 1   raceId           27058 non-null  int64  
 2   driverId         27058 non-null  int64  
 3   constructorId_x  27058 non-null  int64  
 4   number_x         27058 non-null  object 
 5   grid             27058 non-null  int64  
 6   position_x       27058 non-null  object 
 7   positionText     27058 non-null  object 
 8   positionOrder    27058 non-null  int64  
 9   points           27058 non-null  float64
 10  laps             27058 non-null  int64  
 11  time_x           27058 non-null  object 
 12  milliseconds     27058 non-null  object 
 13  fastestLap       27058 non-null  object 
 14  rank             27058 non-null  object 
 15  fastestLapTime   27058 non-null  object 
 16  fastestLapSpeed  27058 non-null  object 
 17  statusId    

In [129]:
df.drop(['url_x',
         'url_y',
         'url',
         'dob',
         'fp1_date',
         'fp2_date',
         'fp3_date',
         'quali_date',
         'sprint_date',
         'sprint_time',
         'name_y',
         'constructorId_y',
         'forename',
         'number_x',
         'number_y',
         'position_y',
         'laps',
         'surname'],

axis=1, inplace=True)
df.head()

,resultId,raceId,driverId,constructorId_x,grid,position_x,positionText,positionOrder,points,time_x,...,code,nationality_x,constructorRef,nationality_y,status,qualifyId,number,q1,q2,q3
0,1,18,1,1,1,1,1,1,10.0,1:34:50.616,...,HAM,British,mclaren,British,Finished,1.0,22.0,1:26.572,1:25.187,1:26.714
1,2,18,2,2,5,2,2,2,8.0,+5.478,...,HEI,German,bmw_sauber,German,Finished,5.0,3.0,1:25.960,1:25.518,1:27.236
2,3,18,3,3,7,3,3,3,6.0,+8.163,...,ROS,German,williams,British,Finished,7.0,7.0,1:26.295,1:26.059,1:28.687
3,4,18,4,4,11,4,4,4,5.0,+17.181,...,ALO,Spanish,renault,French,Finished,12.0,5.0,1:26.907,1:26.188,\N
4,5,18,5,1,3,5,5,5,4.0,+18.014,...,KOV,Finnish,mclaren,British,Finished,3.0,23.0,1:25.664,1:25.452,1:27.079


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27058 entries, 0 to 27057
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         27058 non-null  int64  
 1   raceId           27058 non-null  int64  
 2   driverId         27058 non-null  int64  
 3   constructorId_x  27058 non-null  int64  
 4   grid             27058 non-null  int64  
 5   position_x       27058 non-null  object 
 6   positionText     27058 non-null  object 
 7   positionOrder    27058 non-null  int64  
 8   points           27058 non-null  float64
 9   time_x           27058 non-null  object 
 10  milliseconds     27058 non-null  object 
 11  fastestLap       27058 non-null  object 
 12  rank             27058 non-null  object 
 13  fastestLapTime   27058 non-null  object 
 14  fastestLapSpeed  27058 non-null  object 
 15  statusId         27058 non-null  int64  
 16  year             27058 non-null  int64  
 17  round       

In [131]:
import re

# Combining to just see the quali lap times, gets rid of a lot of the N/A

# helper: "M:SS.sss" or "SS.sss" -> float seconds
pat = re.compile(r"^(?:(\d+):)?(\d+(?:\.\d+)?)$")
def lap_to_seconds(x):
    if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
    s = str(x).strip()
    m = pat.match(s)
    if not m: return np.nan
    mm = int(m.group(1)) if m.group(1) else 0
    ss = float(m.group(2))
    return mm*60 + ss

# 1) parse times
for c in ["q1","q2","q3"]:
    if c in df.columns:
        df[c+"_s"] = df[c].apply(lap_to_seconds)

# 2) best quali lap (skips NaN automatically)
keep_cols = [c for c in ["q1_s","q2_s","q3_s"] if c in df.columns]
df["quali_best_s"] = df[keep_cols].min(axis=1, skipna=True)

# 3) flag: no recorded quali lap
df["no_quali_time"] = df[keep_cols].isna().all(axis=1).astype(int)

# 4) optional: impute worst-case for strict models
if df["quali_best_s"].isna().any():
    worst = df["quali_best_s"].max()
    df["quali_best_s_filled"] = df["quali_best_s"].fillna(worst + 10.0)  # 10s slower than worst seen

# 5) drop originals
df = df.drop(columns=[c for c in ["q1","q2","q3","q1_s","q2_s","q3_s"] if c in df.columns])



In [132]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27058 entries, 0 to 27057
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   resultId             27058 non-null  int64  
 1   raceId               27058 non-null  int64  
 2   driverId             27058 non-null  int64  
 3   constructorId_x      27058 non-null  int64  
 4   grid                 27058 non-null  int64  
 5   position_x           27058 non-null  object 
 6   positionText         27058 non-null  object 
 7   positionOrder        27058 non-null  int64  
 8   points               27058 non-null  float64
 9   time_x               27058 non-null  object 
 10  milliseconds         27058 non-null  object 
 11  fastestLap           27058 non-null  object 
 12  rank                 27058 non-null  object 
 13  fastestLapTime       27058 non-null  object 
 14  fastestLapSpeed      27058 non-null  object 
 15  statusId             27058 non-null 

In [133]:
df.drop(['name_x',
         'number',
         'position_x',
         'rank',
         'date',
         'fastestLapTime',
         'fastestLapSpeed',
         'time_y',
         'time_x',
         'code'],

axis=1, inplace=True)
df.head()

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,points,milliseconds,fastestLap,...,quali_time,driverRef,nationality_x,constructorRef,nationality_y,status,qualifyId,quali_best_s,no_quali_time,quali_best_s_filled
0,1,18,1,1,1,1,1,10.0,5690616,39,...,\N,hamilton,British,mclaren,British,Finished,1.0,85.187,0,85.187
1,2,18,2,2,5,2,2,8.0,5696094,41,...,\N,heidfeld,German,bmw_sauber,German,Finished,5.0,85.518,0,85.518
2,3,18,3,3,7,3,3,6.0,5698779,41,...,\N,rosberg,German,williams,British,Finished,7.0,86.059,0,86.059
3,4,18,4,4,11,4,4,5.0,5707797,58,...,\N,alonso,Spanish,renault,French,Finished,12.0,86.188,0,86.188
4,5,18,5,1,3,5,5,4.0,5708630,43,...,\N,kovalainen,Finnish,mclaren,British,Finished,3.0,85.452,0,85.452


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27058 entries, 0 to 27057
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   resultId             27058 non-null  int64  
 1   raceId               27058 non-null  int64  
 2   driverId             27058 non-null  int64  
 3   constructorId_x      27058 non-null  int64  
 4   grid                 27058 non-null  int64  
 5   positionText         27058 non-null  object 
 6   positionOrder        27058 non-null  int64  
 7   points               27058 non-null  float64
 8   milliseconds         27058 non-null  object 
 9   fastestLap           27058 non-null  object 
 10  statusId             27058 non-null  int64  
 11  year                 27058 non-null  int64  
 12  round                27058 non-null  int64  
 13  circuitId            27058 non-null  int64  
 14  fp1_time             26759 non-null  object 
 15  fp2_time             26759 non-null 

In [135]:
df.drop(['milliseconds',
         'fastestLap',
         'quali_time',
         'nationality_y',
         'points'],

axis=1, inplace=True)

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27058 entries, 0 to 27057
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   resultId             27058 non-null  int64  
 1   raceId               27058 non-null  int64  
 2   driverId             27058 non-null  int64  
 3   constructorId_x      27058 non-null  int64  
 4   grid                 27058 non-null  int64  
 5   positionText         27058 non-null  object 
 6   positionOrder        27058 non-null  int64  
 7   statusId             27058 non-null  int64  
 8   year                 27058 non-null  int64  
 9   round                27058 non-null  int64  
 10  circuitId            27058 non-null  int64  
 11  fp1_time             26759 non-null  object 
 12  fp2_time             26759 non-null  object 
 13  fp3_time             26759 non-null  object 
 14  driverRef            27058 non-null  object 
 15  nationality_x        27058 non-null 

In [137]:
df.tail()

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,statusId,year,round,...,fp2_time,fp3_time,driverRef,nationality_x,constructorRef,status,qualifyId,quali_best_s,no_quali_time,quali_best_s_filled
27053,27059,1159,863,131,11,16,16,1,2025,15,...,NaN,NaN,antonelli,Italian,mercedes,Finished,10842.0,69.493,0,69.493
27054,27060,1159,842,214,14,17,17,1,2025,15,...,NaN,NaN,gasly,French,alpine,Finished,10845.0,69.637,0,69.637
27055,27061,1159,846,1,2,18,18,44,2025,15,...,NaN,NaN,norris,British,mclaren,Oil leak,10833.0,68.674,0,68.674
27056,27062,1159,844,6,6,R,19,4,2025,15,...,NaN,NaN,leclerc,Monegasque,ferrari,Collision,10837.0,69.304,0,69.304
27057,27063,1159,1,6,7,R,20,3,2025,15,...,NaN,NaN,hamilton,British,ferrari,Accident,10838.0,69.261,0,69.261


In [138]:
df.drop(['fp1_time', 'fp2_time', 'fp3_time'], axis=1, inplace=True)

In [139]:
df.head(5)

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,statusId,year,round,circuitId,driverRef,nationality_x,constructorRef,status,qualifyId,quali_best_s,no_quali_time,quali_best_s_filled
0,1,18,1,1,1,1,1,1,2008,1,1,hamilton,British,mclaren,Finished,1.0,85.187,0,85.187
1,2,18,2,2,5,2,2,1,2008,1,1,heidfeld,German,bmw_sauber,Finished,5.0,85.518,0,85.518
2,3,18,3,3,7,3,3,1,2008,1,1,rosberg,German,williams,Finished,7.0,86.059,0,86.059
3,4,18,4,4,11,4,4,1,2008,1,1,alonso,Spanish,renault,Finished,12.0,86.188,0,86.188
4,5,18,5,1,3,5,5,1,2008,1,1,kovalainen,Finnish,mclaren,Finished,3.0,85.452,0,85.452


In [140]:
subRecent_results = df[df['year'] >= 2010]
subRecent_results.head(40)

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,statusId,year,round,circuitId,driverRef,nationality_x,constructorRef,status,qualifyId,quali_best_s,no_quali_time,quali_best_s_filled
20320,20323,337,4,6,3,1,1,1,2010,1,3,alonso,Spanish,ferrari,Finished,3328.0,114.172,0,114.172
20321,20324,337,13,6,2,2,2,1,2010,1,3,massa,Brazilian,ferrari,Finished,3327.0,114.242,0,114.242
20322,20325,337,1,1,4,3,3,1,2010,1,3,hamilton,British,mclaren,Finished,3329.0,114.707,0,114.707
20323,20326,337,20,9,1,4,4,1,2010,1,3,vettel,German,red_bull,Finished,3326.0,113.883,0,113.883
20324,20327,337,3,131,5,5,5,1,2010,1,3,rosberg,German,mercedes,Finished,3330.0,114.682,0,114.682
20325,20328,337,30,131,7,6,6,1,2010,1,3,michael_schumacher,German,mercedes,Finished,3332.0,115.105,0,115.105
20326,20329,337,18,1,8,7,7,1,2010,1,3,button,British,mclaren,Finished,3333.0,115.168,0,115.168
20327,20330,337,17,9,6,8,8,1,2010,1,3,webber,Australian,red_bull,Finished,3331.0,114.318,0,114.318
20328,20331,337,24,10,12,9,9,1,2010,1,3,liuzzi,Italian,force_india,Finished,3337.0,115.623,0,115.623
20329,20332,337,22,3,11,10,10,1,2010,1,3,barrichello,Brazilian,williams,Finished,3336.0,115.330,0,115.330


In [141]:
subRecent_results = subRecent_results.sort_values(by=['year', 'round'])
subRecent_results.tail()

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,statusId,year,round,circuitId,driverRef,nationality_x,constructorRef,status,qualifyId,quali_best_s,no_quali_time,quali_best_s_filled
27053,27059,1159,863,131,11,16,16,1,2025,15,39,antonelli,Italian,mercedes,Finished,10842.0,69.493,0,69.493
27054,27060,1159,842,214,14,17,17,1,2025,15,39,gasly,French,alpine,Finished,10845.0,69.637,0,69.637
27055,27061,1159,846,1,2,18,18,44,2025,15,39,norris,British,mclaren,Oil leak,10833.0,68.674,0,68.674
27056,27062,1159,844,6,6,R,19,4,2025,15,39,leclerc,Monegasque,ferrari,Collision,10837.0,69.304,0,69.304
27057,27063,1159,1,6,7,R,20,3,2025,15,39,hamilton,British,ferrari,Accident,10838.0,69.261,0,69.261


In [142]:
subRecent_results['status'].value_counts()

,count
status,
Finished,3509
+1 Lap,1573
+2 Laps,344
Collision,244
Accident,145
...,...
Spark plugs,1
Water pump,1
Cooling system,1


In [143]:
# crash/relaibilty predictor NOT FINISHED

subRecent_results["accident_flag"] = ((subRecent_results["status"] == "Accident") | (subRecent_results["status"] == "Spun off")).astype(int)

subRecent_results["driverErrorRate"] = (
    subRecent_results.groupby("driverRef")["accident_flag"].transform("mean")
)

#subRecent_results = subRecent_results.sort_values(by=['driver', 'year', 'round'])
#df['driverErrorRate'] = subRecent_results.groupby('driver')['status'].apply(lambda x: (x == 'Accident').mean())

In [144]:
# Average Race Position - Simulating Driver's Racing

avg_pos = df.groupby("driverRef")["positionOrder"].mean()

avg_pos_score = 100 * (avg_pos.max() - avg_pos) / (avg_pos.max() - avg_pos.min())

subRecent_results['driverScore'] = subRecent_results['driverRef'].map(avg_pos_score)

In [145]:
#momentum feature with different race intervals

subRecent_results["pos_clean"] = np.where(
    subRecent_results["positionText"].isin(["R", "D"]),
    np.nan,
    subRecent_results["positionOrder"]
)

for window in [3, 5, 10]:
    subRecent_results[f"momentum_{window}"] = subRecent_results.groupby("driverRef")["pos_clean"].transform(
        lambda x: -x.rolling(window, min_periods=1).mean()
    )

In [146]:
# Team's average position this season
subRecent_results["constructor_avg_pos"] = subRecent_results.groupby(["constructorRef", "year"])["positionOrder"].transform("mean")

# Team's momentum (improving/declining car)
subRecent_results["constructor_momentum"] = subRecent_results.groupby("constructorRef")["positionOrder"].transform(
    lambda x: -x.rolling(5, min_periods=1).mean()
)

In [147]:
# How well does this driver perform at THIS circuit? (Exponentially weighted)
# We'll use expand=False to only use data up to the current row for the calculation
subRecent_results["driver_circuit_avg"] = subRecent_results.groupby(["driverRef", "circuitId"])["positionOrder"].transform(
    lambda x: x.ewm(span=5, adjust=False).mean()
)

# How well does this constructor perform at THIS circuit? (Exponentially weighted)
subRecent_results["constructor_circuit_avg"] = subRecent_results.groupby(["constructorRef", "circuitId"])["positionOrder"].transform(
    lambda x: x.ewm(span=5, adjust=False).mean()
)

In [148]:
subRecent_results.tail(5)

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,statusId,year,round,...,driverErrorRate,driverScore,pos_clean,momentum_3,momentum_5,momentum_10,constructor_avg_pos,constructor_momentum,driver_circuit_avg,constructor_circuit_avg
27053,27059,1159,863,131,11,16,16,1,2025,15,...,0.000000,73.936652,16.0,-14.000000,-14.00,-11.500000,8.166667,-9.8,16.000000,9.728598
27054,27060,1159,842,214,14,17,17,1,2025,15,...,0.000000,74.280436,17.0,-15.333333,-13.00,-12.666667,14.766667,-16.8,9.987654,12.665905
27055,27061,1159,846,1,2,18,18,44,2025,15,...,0.006993,85.210265,18.0,-7.000000,-4.60,-4.800000,3.166667,-4.8,9.925926,8.850683
27056,27062,1159,844,6,6,R,19,4,2025,15,...,0.024390,85.084428,NaN,-3.500000,-6.00,-5.222222,7.566667,-9.0,11.098765,10.865264
27057,27063,1159,1,6,7,R,20,3,2025,15,...,0.006270,91.445403,NaN,-9.500000,-6.75,-6.222222,7.566667,-12.4,10.123457,13.910176


In [149]:
subRecent_results["momentum_3"].isna().sum()

np.int64(57)

In [150]:
#switching for final csv

finaldf = subRecent_results

In [151]:
finaldf.head()
finaldf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6735 entries, 20320 to 27057
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   resultId                 6735 non-null   int64  
 1   raceId                   6735 non-null   int64  
 2   driverId                 6735 non-null   int64  
 3   constructorId_x          6735 non-null   int64  
 4   grid                     6735 non-null   int64  
 5   positionText             6735 non-null   object 
 6   positionOrder            6735 non-null   int64  
 7   statusId                 6735 non-null   int64  
 8   year                     6735 non-null   int64  
 9   round                    6735 non-null   int64  
 10  circuitId                6735 non-null   int64  
 11  driverRef                6735 non-null   object 
 12  nationality_x            6735 non-null   object 
 13  constructorRef           6735 non-null   object 
 14  status                  

In [152]:
final2df = finaldf

In [153]:
final2df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6735 entries, 20320 to 27057
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   resultId                 6735 non-null   int64  
 1   raceId                   6735 non-null   int64  
 2   driverId                 6735 non-null   int64  
 3   constructorId_x          6735 non-null   int64  
 4   grid                     6735 non-null   int64  
 5   positionText             6735 non-null   object 
 6   positionOrder            6735 non-null   int64  
 7   statusId                 6735 non-null   int64  
 8   year                     6735 non-null   int64  
 9   round                    6735 non-null   int64  
 10  circuitId                6735 non-null   int64  
 11  driverRef                6735 non-null   object 
 12  nationality_x            6735 non-null   object 
 13  constructorRef           6735 non-null   object 
 14  status                  

In [154]:
y = final2df["positionOrder"]   # lower = better
X = final2df[["grid","quali_best_s_filled","momentum_3","momentum_5","momentum_10",
             "driverErrorRate","driverScore",
              "constructor_avg_pos","constructor_momentum",
              "driver_circuit_avg","constructor_circuit_avg",
             "driverRef","constructorId_x","circuitId"]]
X = pd.get_dummies(X, columns=["driverRef","constructorId_x","circuitId"], drop_first=True)


In [155]:
cutoff = 7

# Training: only 2025 up to cutoff round
train = finaldf[(finaldf["year"] == 2025) & (finaldf["round"] <= cutoff)]

# Testing: only 2025 after cutoff round
test  = finaldf[(finaldf["year"] == 2025) & (finaldf["round"] > cutoff)]

X_train, y_train = X.loc[train.index], y.loc[train.index]
X_test, y_test   = X.loc[test.index], y.loc[test.index]

In [156]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import spearmanr

models = {
    "RandomForest": RandomForestRegressor(n_estimators=500, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
}

for name, model  in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    spearman, pval = spearmanr(y_test, preds)

    print(f"{name} → MAE: {mae:.2f}, RMSE: {rmse:.2f}")
    print("------")
    print(f"Spearman correlation: {spearman:.3f} (p={pval:.3f})")


RandomForest → MAE: 2.05, RMSE: 2.81
------
Spearman correlation: 0.877 (p=0.000)
XGBoost → MAE: 2.23, RMSE: 3.03
------
Spearman correlation: 0.840 (p=0.000)


In [161]:
final2df.to_csv('cleanedData3.csv', index=False)

In [158]:
final2df.tail(10)

,resultId,raceId,driverId,constructorId_x,grid,positionText,positionOrder,statusId,year,round,...,driverErrorRate,driverScore,pos_clean,momentum_3,momentum_5,momentum_10,constructor_avg_pos,constructor_momentum,driver_circuit_avg,constructor_circuit_avg
27048,27054,1159,861,214,16,11,11,1,2025,15,...,0.055556,65.912519,11.0,-16.000000,-16.60,-15.555556,14.766667,-15.4,11.000000,10.498857
27049,27055,1159,859,215,8,12,12,1,2025,15,...,0.038462,70.344588,12.0,-9.333333,-8.50,-9.571429,12.300000,-10.8,12.666667,10.740741
27050,27056,1159,832,3,9,13,13,1,2025,15,...,0.026906,79.546700,13.0,-15.000000,-15.40,-12.800000,11.600000,-13.0,8.358025,12.263730
27051,27057,1159,807,15,17,14,14,1,2025,15,...,0.032653,73.050143,14.0,-13.000000,-10.20,-10.600000,13.200000,-10.8,12.444444,17.555556
27052,27058,1159,864,15,13,15,15,1,2025,15,...,0.133333,66.153846,15.0,-10.000000,-9.50,-12.000000,13.200000,-12.0,15.000000,16.703704
27053,27059,1159,863,131,11,16,16,1,2025,15,...,0.000000,73.936652,16.0,-14.000000,-14.00,-11.500000,8.166667,-9.8,16.000000,9.728598
27054,27060,1159,842,214,14,17,17,1,2025,15,...,0.000000,74.280436,17.0,-15.333333,-13.00,-12.666667,14.766667,-16.8,9.987654,12.665905
27055,27061,1159,846,1,2,18,18,44,2025,15,...,0.006993,85.210265,18.0,-7.000000,-4.60,-4.800000,3.166667,-4.8,9.925926,8.850683
27056,27062,1159,844,6,6,R,19,4,2025,15,...,0.024390,85.084428,NaN,-3.500000,-6.00,-5.222222,7.566667,-9.0,11.098765,10.865264
27057,27063,1159,1,6,7,R,20,3,2025,15,...,0.006270,91.445403,NaN,-9.500000,-6.75,-6.222222,7.566667,-12.4,10.123457,13.910176


In [159]:
# f1modelq.py (fixed bits)
from xgboost import XGBRegressor
import pandas as pd

df = pd.read_csv('/content/cleanedData.csv')

WITH_QUALI = [
    "grid","quali_best_s_filled",
    "momentum_3","momentum_5","momentum_10",
    "driverErrorRate","driverScore",
    "constructor_avg_pos","constructor_momentum",
    "driver_circuit_avg","constructor_circuit_avg",
    "driverRef","constructorId_x","circuitId"
]
WITHOUT_QUALI = [
    "momentum_3","momentum_5","momentum_10",
    "driverErrorRate","driverScore",
    "constructor_avg_pos","constructor_momentum",
    "driver_circuit_avg","constructor_circuit_avg",
    "driverRef","constructorId_x","circuitId"
]

def train2Model(df):
    y = df["positionOrder"]

    Xq = pd.get_dummies(df[WITH_QUALI], columns=["driverRef","constructorId_x","circuitId"], drop_first=True)
    Xn = pd.get_dummies(df[WITHOUT_QUALI], columns=["driverRef","constructorId_x","circuitId"], drop_first=True)

    train_idx = df["year"] < 2023

    modelQ = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
    modelQ.fit(Xq.loc[train_idx], y.loc[train_idx])

    modelN = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=42)
    modelN.fit(Xn.loc[train_idx], y.loc[train_idx])

    return {
        "with_quali": {"model": modelQ, "columns": Xq.columns},
        "no_quali":   {"model": modelN, "columns": Xn.columns},
    }

artifacts = train2Model(df)


FileNotFoundError: [Errno 2] No such file or directory: '/content/cleanedData.csv'

In [ ]:
# choose the upcoming context
UPCOMING_YEAR  = 2025
UPCOMING_ROUND = 16         # set the next round number
UPCOMING_CID   = 14         # circuitId for that GP (use your circuits mapping)

snap = build_snapshot(df, UPCOMING_YEAR, UPCOMING_ROUND, UPCOMING_CID)
pre_table = predict_pre_quali(snap, artifacts)

print(pre_table.head(20))   # <-- sorted 1..20 projection


In [ ]:
quali = {
    "leclerc": {"grid": 1, "quali_best_s_filled": 70.812},
    "verstappen": {"grid": 2},
    # ... include all drivers you have
}

post_table = predict_post_quali(snap, artifacts, quali)
print(post_table.head(20))  # <-- sorted 1..20 projection
